In [1]:
import navis
import fafbseg
import pymaid

In [2]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [3]:
from pyroglancer.layers import create_nglayer
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.ngviewer import openviewer, closeviewer
from pyroglancer.ngspaces import create_ngspace

# Step 1: Fetch stuff from flywire..

In [4]:
#set the flywire token here..
#fafbseg.flywire.set_chunkedgraph_secret("blabla")

In [5]:
#mesh in flywire for which to find a match..
flywire_id = 720575940633919863

In [6]:
# Fetch the neuron's mesh
putativeDNa02_a = fafbseg.flywire.get_mesh_neuron(flywire_id)
putativeDNa02_a

,
type,navis.MeshNeuron
name,None
id,720575940633919863
units,1 nanometer
n_vertices,785994
n_faces,1575396


# Step 2: Transform it to fafb..

In [7]:
DNa02_xyz = pd.DataFrame(list(map(np.ravel, putativeDNa02_a.vertices)), columns=['x', 'y', 'z'])

In [8]:
#move the vertices of the flywire mesh to fafb..
fafb_putativeDNa02_xyz = navis.xform_brain(DNa02_xyz, source='FLYWIRE', target='FAFB14')

Transform path: FLYWIRE -> FLYWIREraw -> FAFB14raw -> FAFB14


In [9]:
fafb_putativeDNa02_a = putativeDNa02_a.copy()

In [10]:
#put the transformed vertices in the new mesh..
fafb_putativeDNa02_a.vertices = fafb_putativeDNa02_xyz.to_numpy()

In [11]:
#perform a bit of conversion to navis volumes..
fafb_putativeDNa02_a = navis.core.volumes.Volume(fafb_putativeDNa02_a)
fafb_putativeDNa02_a.id = flywire_id
fafb_putativeDNa02_a.name = 'putative_DNa02'
fafb_putativeDNa02_a

<navis.Volume(name=putative_DNa02, id=720575940633919863, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(785994, 3), faces.shape=(1575396, 3))>

# Step 3: fetch neuropils from catmaid for display..

In [12]:
publicurl = 'https://fafb.catmaid.virtualflybrain.org/'

In [13]:
working_rm = pymaid.CatmaidInstance(publicurl, api_token=None, project_id = 1)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


In [14]:
# #set up catmaid (in case of private)..
# working_rm = pymaid.CatmaidInstance('https://neuropil.janelia.org/tracing/fafb/v14',
#                             api_token=os.environ['TOKEN'],
#                             http_user=os.environ['authname'], # omit if not required
#                             http_password=os.environ['authpassword'], project_id = 1) # omit if not required

In [15]:
#fetch the LAL neuropil mesh..
LAL_neuropil = pymaid.get_volume('LAL_L', color=(255, 0, 0, .2))

In [16]:
#perform a bit of conversion to navis volumes..
LAL_neuropil = navis.core.volumes.Volume(LAL_neuropil)
LAL_neuropil.id = 100
LAL_neuropil.name = 'LAL_L'
LAL_neuropil

<navis.Volume(name=LAL_L, id=100, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(402, 3), faces.shape=(800, 3))>

In [17]:
#fetch the FB neuropil mesh..
FB_neuropil = pymaid.get_volume('FB', color=(0, 255, 0, .2))

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


In [18]:
#perform a bit of conversion to navis volumes..
FB_neuropil = navis.core.volumes.Volume(FB_neuropil)
FB_neuropil.id = 200
FB_neuropil.name = 'FB'
FB_neuropil

<navis.Volume(name=FB, id=200, color=(0.85, 0.85, 0.85, 0.2), vertices.shape=(614, 3), faces.shape=(1224, 3))>

# Step 4: Set up pyroglancer configuration..

In [19]:
startdataserver()

Serving data from:  /var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u
Serving directory at http://127.0.0.1:8000


In [20]:
create_ngspace('FAFB')

using default location at:  /Users/sri/.pyroglancer/config.yml
Neuroglancer viewer created at:  http://127.0.0.1:56945/v/576c0ea22c80d42cbea5432f2c43044b609cfa37/
using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  image
using default location at:  /Users/sri/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  image
using default location at:  /Users/sri/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  surfacemesh
using default location at:  /Users/sri/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  segmentation
using default location at:  /Users/sri/.pyroglancer/config.yml
Using layout : xy-3d
using default location at:  /Users/sri/.pyroglan

# Step 5: Inject the neuron mesh and neuropils..

In [21]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': fafb_putativeDNa02_a,
                                        'name': 'flywiremeshes','color': 'yellow'})

using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/720575940633919863
Seg id is: 720575940633919863
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/flywiremeshes/mesh/720575940633919863
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/flywiremeshes/mesh/segment_properties
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/flywiremeshes/mesh/segment_names
Using layout : xy-3d


In [22]:
tmpviewer = create_nglayer(layer_kws = {'type': 'volumes','source': [LAL_neuropil,FB_neuropil],
                                        'name': 'neuropils','color': ['magenta', 'cyan'], 'alpha': 0.3})

using default location at:  /Users/sri/.pyroglancer/config.yml
Dimensions are in : FAFB
Layer created:  volumes
mesh/100
Seg id is: 100
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/neuropils/mesh/100
mesh/200
Seg id is: 200
Full filepath: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/neuropils/mesh/200
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/neuropils/mesh/segment_properties
creating: /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u/precomputed/neuropils/mesh/segment_names
Using layout : xy-3d


127.0.0.1 - - [26/Feb/2021 13:33:47] "GET /precomputed/flywiremeshes/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:33:47] "GET /precomputed/neuropils/mesh/info HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:33:47] "GET /precomputed/flywiremeshes/mesh/segment_properties/info HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:33:47] "GET /precomputed/neuropils/mesh/segment_properties/info HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:32] "GET /precomputed/neuropils/mesh/100:0 HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:32] "GET /precomputed/neuropils/mesh/200:0 HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:32] "GET /precomputed/neuropils/mesh/100 HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:32] "GET /precomputed/neuropils/mesh/200 HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:37] "GET /precomputed/flywiremeshes/mesh/720575940633919863:0 HTTP/1.1" 200 -
127.0.0.1 - - [26/Feb/2021 13:34:37] "GET /precomputed/flywiremeshes/mesh/720575940633919863 HTTP/1.1" 200 -
127.0.

# Step 6: After injection poke around the flywire mesh

### The yellow mesh is the flywire neuron mesh and the green mesh is the fafb poke around match. The magenta region is the LAL, and the cyan region is the FB. The red dots are presynapses, the blue dots are postsynapses. You can see that the putativeDNa02_a is indeed getting many synapses from LAL

![alt text](pics/flywire2fafb.png "Title")

In [23]:
closeviewer()

closing already existing ng viewer


In [24]:
closedataserver()

Closing server at http://127.0.0.1:8000
Cleaning directory at /private/var/folders/_l/lrfvj_8j3ps0c37ncbr3c8dh0000gn/T/tmpspux6y4u
